<a href="https://colab.research.google.com/github/AkshatShrinate/Zeotap/blob/main/Akshat_Shrinate_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

#Uploaded the files in a public github reporsitory and then imported the file into Google Colab
customer_dataset = pd.read_csv('https://raw.githubusercontent.com/AkshatShrinate/Zeotap/main/Datasets/Customers.csv')
Products_dataset = pd.read_csv('https://raw.githubusercontent.com/AkshatShrinate/Zeotap/main/Datasets/Products.csv')
Transactions_dataset = pd.read_csv('https://raw.githubusercontent.com/AkshatShrinate/Zeotap/main/Datasets/Transactions.csv')

In [2]:
customer_dataset.head()

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15


In [3]:
Products_dataset.head()

,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31


In [4]:
Transactions_dataset.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 7:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


In [6]:
#to check for missing data values
print(customer_dataset.isnull().sum())
print(Products_dataset.isnull().sum())
print(Transactions_dataset.isnull().sum())

CustomerID      0
CustomerName    0
Region          0
SignupDate      0
dtype: int64
ProductID      0
ProductName    0
Category       0
Price          0
dtype: int64
TransactionID      0
CustomerID         0
ProductID          0
TransactionDate    0
Quantity           0
TotalValue         0
Price              0
dtype: int64


In [26]:
#to check for duplicated rows
print(customer_dataset.duplicated().sum())
print(Products_dataset.duplicated().sum())
print(Transactions_dataset.duplicated().sum())

0
0
0


In [27]:
#A merge dataset is obtained by merging the 3 data sets through the unique identifiers Product_ID and Customer_ID. Also the Price column appears twice therefore to reduce redundancy
transactions_customers = Transactions_dataset.merge(customer_dataset, on='CustomerID', how='inner')
merged_data = transactions_customers.merge(Products_dataset, on='ProductID', how='inner')
new_order = ['TransactionDate', 'CustomerID', 'CustomerName', 'Region',
             'ProductID', 'ProductName', 'Category', 'Price_x', 'Quantity', 'TotalValue','TransactionID','SignupDate' ]

# Just to create a dataset with reordered columns
new_dataset = merged_data[new_order]

# Verify the new order
new_dataset=new_dataset.sort_values(by="CustomerID")
new_dataset.head()

,TransactionDate,CustomerID,CustomerName,Region,ProductID,ProductName,Category,Price_x,Quantity,TotalValue,TransactionID,SignupDate
442,2024-09-17 9:01:18,C0001,Lawrence Carroll,South America,P022,HomeSense Wall Art,Home Decor,137.54,3,412.62,T00932,2022-07-10
724,2024-05-07 3:11:44,C0001,Lawrence Carroll,South America,P083,ActiveWear Smartwatch,Electronics,455.72,2,911.44,T00445,2022-07-10
135,2024-01-19 3:12:55,C0001,Lawrence Carroll,South America,P054,SoundWave Cookbook,Books,57.30,2,114.60,T00015,2022-07-10
776,2024-11-02 17:04:16,C0001,Lawrence Carroll,South America,P029,TechPro Headphones,Electronics,433.64,3,1300.92,T00436,2022-07-10
546,2024-04-08 0:01:00,C0001,Lawrence Carroll,South America,P096,SoundWave Headphones,Electronics,307.47,2,614.94,T00085,2022-07-10


In [28]:
new_dataset.rename(columns={'Price_x': 'Product_Price'}, inplace=True) #rename the column name

In [29]:
new_dataset.head()

,TransactionDate,CustomerID,CustomerName,Region,ProductID,ProductName,Category,Product_Price,Quantity,TotalValue,TransactionID,SignupDate
442,2024-09-17 9:01:18,C0001,Lawrence Carroll,South America,P022,HomeSense Wall Art,Home Decor,137.54,3,412.62,T00932,2022-07-10
724,2024-05-07 3:11:44,C0001,Lawrence Carroll,South America,P083,ActiveWear Smartwatch,Electronics,455.72,2,911.44,T00445,2022-07-10
135,2024-01-19 3:12:55,C0001,Lawrence Carroll,South America,P054,SoundWave Cookbook,Books,57.30,2,114.60,T00015,2022-07-10
776,2024-11-02 17:04:16,C0001,Lawrence Carroll,South America,P029,TechPro Headphones,Electronics,433.64,3,1300.92,T00436,2022-07-10
546,2024-04-08 0:01:00,C0001,Lawrence Carroll,South America,P096,SoundWave Headphones,Electronics,307.47,2,614.94,T00085,2022-07-10


Feature Engineering is used to identify similar customers who might be grouped together. Using the new_dataset, a few parameters are identified to identify Customers who are different from other customers. Thus, a Lookalike model can be built based on these parameters.

Identifed Parameteres include :

1) Total transactions from each customer

2) Total spending by each customer

3) The Average spending by each customer

4) The Top Category for each customer

5) Duration for which the customer is active

6) Region of each customer

Using these parameters, a customer_dataset is created which is then used to train a Lookalike model that provides top 3 similar customers.

In [30]:
'''1) Total transactions from each customer'''
# To group and give a total count of transactions per customer
customer_records = Transactions_dataset.groupby('CustomerID')['TransactionID'].count().reset_index()
customer_records.columns = ['CustomerID', 'TotalTransactions']
print(customer_records.head())

  CustomerID  TotalTransactions
0      C0001                  5
1      C0002                  4
2      C0003                  4
3      C0004                  8
4      C0005                  3


In [31]:
'''2) Total spending by each customer'''
# Calculate total spending done by each customer
customer_spending =Transactions_dataset.groupby('CustomerID')['Price'].sum().reset_index()
customer_spending.columns = ['CustomerID', 'TotalSpending']
customer_records["TotalSpending"]=customer_spending["TotalSpending"] #adding it to the customer_records dataset
print(customer_records.head())


  CustomerID  TotalTransactions  TotalSpending
0      C0001                  5        1391.67
1      C0002                  4         835.68
2      C0003                  4         782.83
3      C0004                  8        1925.09
4      C0005                  3         874.81


In [32]:
'''3) The Average spending by each customer'''
customer_records["AvgSpending"]=customer_records['TotalSpending']/customer_records['TotalTransactions']
customer_records.head()

,CustomerID,TotalTransactions,TotalSpending,AvgSpending
0,C0001,5,1391.67,278.334000
1,C0002,4,835.68,208.920000
2,C0003,4,782.83,195.707500
3,C0004,8,1925.09,240.636250
4,C0005,3,874.81,291.603333


In [33]:
'''4) The Top Category of product bought by each customer'''
#First group the transactions by customer ID then group orders made by each customer by using the category to get a total count of each category for a user
top_category = new_dataset.groupby(['CustomerID', 'Category'])['TransactionID'].count().reset_index()
'''Now sorting is done by Customer ID in ASC order then by number of transactions for each category in descending order'''
top_category = top_category.sort_values(['CustomerID', 'TransactionID'], ascending=[True, False])
top_category.head()


,CustomerID,Category,TransactionID
1,C0001,Electronics,3
0,C0001,Books,1
2,C0001,Home Decor,1
3,C0002,Clothing,2
4,C0002,Home Decor,2


In [34]:
# We have received the transactions of each user in each category but we want only the top most one as it represents the top most category for each user
top_category = top_category.drop_duplicates(subset=['CustomerID'], keep='first')[['CustomerID', 'Category']]
top_category.columns = ['CustomerID', 'TopCategory']
customer_records["Top_Category"]=top_category['TopCategory']
print(top_category.head())

   CustomerID  TopCategory
1       C0001  Electronics
3       C0002     Clothing
7       C0003   Home Decor
8       C0004        Books
11      C0005  Electronics


In [35]:
customer_records1=['CustomerID', 'TotalTransactions', 'TotalSpending', 'AvgSpending']
customer_records=customer_records[customer_records1]
customer_records.head()

,CustomerID,TotalTransactions,TotalSpending,AvgSpending
0,C0001,5,1391.67,278.334000
1,C0002,4,835.68,208.920000
2,C0003,4,782.83,195.707500
3,C0004,8,1925.09,240.636250
4,C0005,3,874.81,291.603333


In [36]:
'''5) Duration for which the customer is active: Calculated using the Signup Date column'''
customer_duration = customer_dataset[['CustomerID', 'SignupDate']]
customer_duration['SignupDate'] = pd.to_datetime(customer_duration['SignupDate'])
customer_duration['Duration_of_Activity'] = (pd.to_datetime('today') - customer_duration['SignupDate']).dt.days #to calculate count of days from Signup Date till today
customer_records=customer_records.merge(customer_duration,on='CustomerID',how='inner') #merging it to the customer_records dataset
customer_records.head()

<ipython-input-36-fbf869f8a337>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_duration['SignupDate'] = pd.to_datetime(customer_duration['SignupDate'])
<ipython-input-36-fbf869f8a337>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_duration['Duration_of_Activity'] = (pd.to_datetime('today') - customer_duration['SignupDate']).dt.days #to calculate count of days from Signup Date till today


,CustomerID,TotalTransactions,TotalSpending,AvgSpending,SignupDate,Duration_of_Activity
0,C0001,5,1391.67,278.334000,2022-07-10,938
1,C0002,4,835.68,208.920000,2022-02-13,1085
2,C0003,4,782.83,195.707500,2024-03-07,332
3,C0004,8,1925.09,240.636250,2022-10-09,847
4,C0005,3,874.81,291.603333,2022-08-15,902


In [37]:
'''6) Region of each customer: To find this, One Hot Encoding is done over the Region column'''
OHE=OneHotEncoder(handle_unknown='ignore',sparse_output=False).set_output(transform='pandas')
ohe=OHE.fit_transform(new_dataset[['Region']]).astype(int)
new_dataset = pd.concat([new_dataset, ohe], axis=1)
# Drop the original Region column from the dataset
new_dataset.drop(columns=['Region'], inplace=True)
new_dataset.head()

,TransactionDate,CustomerID,CustomerName,ProductID,ProductName,Category,Product_Price,Quantity,TotalValue,TransactionID,SignupDate,Region_Asia,Region_Europe,Region_North America,Region_South America
442,2024-09-17 9:01:18,C0001,Lawrence Carroll,P022,HomeSense Wall Art,Home Decor,137.54,3,412.62,T00932,2022-07-10,0,0,0,1
724,2024-05-07 3:11:44,C0001,Lawrence Carroll,P083,ActiveWear Smartwatch,Electronics,455.72,2,911.44,T00445,2022-07-10,0,0,0,1
135,2024-01-19 3:12:55,C0001,Lawrence Carroll,P054,SoundWave Cookbook,Books,57.30,2,114.60,T00015,2022-07-10,0,0,0,1
776,2024-11-02 17:04:16,C0001,Lawrence Carroll,P029,TechPro Headphones,Electronics,433.64,3,1300.92,T00436,2022-07-10,0,0,0,1
546,2024-04-08 0:01:00,C0001,Lawrence Carroll,P096,SoundWave Headphones,Electronics,307.47,2,614.94,T00085,2022-07-10,0,0,0,1


In [38]:
#Similarly now that we have verified the region column's encoding we add it to the customer_records dataset
region_encoded_dataset = new_dataset[['CustomerID'] + list(ohe.columns)]
customer_records = customer_records.merge(region_encoded_dataset, on='CustomerID', how='left') #the merging is done based on the CustomerID
customer_records.head()

,CustomerID,TotalTransactions,TotalSpending,AvgSpending,SignupDate,Duration_of_Activity,Region_Asia,Region_Europe,Region_North America,Region_South America
0,C0001,5,1391.67,278.334,2022-07-10,938,0,0,0,1
1,C0001,5,1391.67,278.334,2022-07-10,938,0,0,0,1
2,C0001,5,1391.67,278.334,2022-07-10,938,0,0,0,1
3,C0001,5,1391.67,278.334,2022-07-10,938,0,0,0,1
4,C0001,5,1391.67,278.334,2022-07-10,938,0,0,0,1


In [42]:
# Merge TopCategory column into customer_records using CustomerID
customer_records = customer_records.merge(top_category, on='CustomerID', how='left')
customer_records.head() #it is noted that after merging a single customerID is displayed multiple times therefore the duplicate data needs to be removed

,CustomerID,TotalTransactions,TotalSpending,AvgSpending,SignupDate,Duration_of_Activity,Region_Asia,Region_Europe,Region_North America,Region_South America,TopCategory
0,C0001,5,1391.67,278.334000,2022-07-10,938,0,0,0,1,Electronics
1,C0002,4,835.68,208.920000,2022-02-13,1085,1,0,0,0,Clothing
2,C0003,4,782.83,195.707500,2024-03-07,332,0,0,0,1,Home Decor
3,C0004,8,1925.09,240.636250,2022-10-09,847,0,0,0,1,Books
4,C0005,3,874.81,291.603333,2022-08-15,902,1,0,0,0,Electronics


In [43]:
customer_records = customer_records.drop_duplicates(subset=['CustomerID'], keep='first')
# Display the cleaned dataset
customer_records.head()

,CustomerID,TotalTransactions,TotalSpending,AvgSpending,SignupDate,Duration_of_Activity,Region_Asia,Region_Europe,Region_North America,Region_South America,TopCategory
0,C0001,5,1391.67,278.334000,2022-07-10,938,0,0,0,1,Electronics
1,C0002,4,835.68,208.920000,2022-02-13,1085,1,0,0,0,Clothing
2,C0003,4,782.83,195.707500,2024-03-07,332,0,0,0,1,Home Decor
3,C0004,8,1925.09,240.636250,2022-10-09,847,0,0,0,1,Books
4,C0005,3,874.81,291.603333,2022-08-15,902,1,0,0,0,Electronics


In [44]:
#The top category also needs one hot encoding to make it easier for the machine to understand it
OHE=OneHotEncoder(handle_unknown='ignore',sparse_output=False).set_output(transform='pandas')
ohe=OHE.fit_transform(customer_records[['TopCategory']]).astype(int)
customer_records = pd.concat([customer_records, ohe], axis=1)

# Drop the original Region column
customer_records.drop(columns=['TopCategory'], inplace=True)
customer_records.head()

,CustomerID,TotalTransactions,TotalSpending,AvgSpending,SignupDate,Duration_of_Activity,Region_Asia,Region_Europe,Region_North America,Region_South America,TopCategory_Books,TopCategory_Clothing,TopCategory_Electronics,TopCategory_Home Decor
0,C0001,5,1391.67,278.334000,2022-07-10,938,0,0,0,1,0,0,1,0
1,C0002,4,835.68,208.920000,2022-02-13,1085,1,0,0,0,0,1,0,0
2,C0003,4,782.83,195.707500,2024-03-07,332,0,0,0,1,0,0,0,1
3,C0004,8,1925.09,240.636250,2022-10-09,847,0,0,0,1,1,0,0,0
4,C0005,3,874.81,291.603333,2022-08-15,902,1,0,0,0,0,0,1,0


In [45]:
print(customer_records.max()) #to check what is the maximum value that each column holds.

CustomerID                               C0200
TotalTransactions                           11
TotalSpending                          3857.06
AvgSpending                             497.76
SignupDate                 2024-12-28 00:00:00
Duration_of_Activity                      1107
Region_Asia                                  1
Region_Europe                                1
Region_North America                         1
Region_South America                         1
TopCategory_Books                            1
TopCategory_Clothing                         1
TopCategory_Electronics                      1
TopCategory_Home Decor                       1
dtype: object


Normalisation of Data: A Min-Max approach is preferred in this situation and the value of the columns is reduced between 0 and 1.

In [46]:
# Create a copy to preserve the original dataset
customer_records_scaled = customer_records.copy()
columns_to_scale = customer_records_scaled.columns.difference(['CustomerID', 'SignupDate'])
# Dropping the 'CustomerID' and 'SignupDate' from the columns_to_scale as they contain non-numeric data
scaler = MinMaxScaler() # Initialize MinMaxScaler
customer_records_scaled[columns_to_scale] = scaler.fit_transform(customer_records_scaled[columns_to_scale])
customer_records_scaled.head()

,CustomerID,TotalTransactions,TotalSpending,AvgSpending,SignupDate,Duration_of_Activity,Region_Asia,Region_Europe,Region_North America,Region_South America,TopCategory_Books,TopCategory_Clothing,TopCategory_Electronics,TopCategory_Home Decor
0,C0001,0.4,0.353913,0.519414,2022-07-10,0.842204,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,C0002,0.3,0.208209,0.367384,2022-02-13,0.979458,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,C0003,0.3,0.194359,0.338446,2024-03-07,0.276377,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,C0004,0.7,0.493703,0.436848,2022-10-09,0.757236,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,C0005,0.2,0.218463,0.548476,2022-08-15,0.808590,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


**Cosine Similarity is found out to create a NxN similarity Matrix which represents a value between 0 and 1 as the percentage of simlarity for each consumer. From this similarity matrix, the top 3 values are then found for each row.**

In [50]:
numeric_data = customer_records.drop(columns=['CustomerID','SignupDate']) # Dropping the 'CustomerID' and 'SignupDate' from the columns_to_scale as they contain non-numeric data
from sklearn.metrics.pairwise import cosine_similarity # This library allows the calculation of the cosine similarity on the numeric columns
similarity_matrix = cosine_similarity(numeric_data) #generates a similarity matrix which is 20x200 as there are 200 customers in the dataset
similarity_dataset = pd.DataFrame(similarity_matrix, index=customer_records['CustomerID'], columns=customer_records['CustomerID']) #to generate a pandas Dataframe
similarity_dataset.head()

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.949973,0.980426,0.983141,0.977333,0.952897,0.986636,0.909421,0.992193,0.952993,...,0.929219,0.997888,0.999187,0.908575,0.883237,0.996734,0.979689,0.853286,0.996804,0.979044
C0002,0.949973,1.000000,0.873013,0.879068,0.990836,0.813356,0.981494,0.735285,0.926246,0.999950,...,0.767837,0.965421,0.952876,0.732744,0.692644,0.926546,0.989078,0.972024,0.969940,0.866748
C0003,0.980426,0.873013,1.000000,0.993613,0.925210,0.993793,0.945865,0.965028,0.989547,0.877801,...,0.981944,0.969964,0.979818,0.967849,0.954316,0.992345,0.930059,0.744284,0.965446,0.998960
C0004,0.983141,0.879068,0.993613,1.000000,0.922210,0.984797,0.941052,0.969378,0.978929,0.883618,...,0.976439,0.969571,0.978330,0.967394,0.950106,0.988954,0.926657,0.744581,0.965496,0.997194
C0005,0.977333,0.990836,0.925210,0.922210,1.000000,0.877713,0.998206,0.802494,0.967204,0.992008,...,0.837116,0.988754,0.981590,0.803511,0.770434,0.964490,0.999912,0.942040,0.991089,0.917703


In [53]:
similarity_dataset= np.floor(similarity_dataset * 100) / 100 #the values of the dataset contain multiple digits after the decimal point
#thus the values are converted to a standard 2 digits after the decimal point value
#Note that Values like 0.999988 are kept as 0.99 and not rounded upto 1.00
similarity_dataset.head()

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.00,0.94,0.98,0.98,0.97,0.95,0.98,0.90,0.99,0.95,...,0.92,0.99,0.99,0.90,0.88,0.99,0.97,0.85,0.99,0.97
C0002,0.94,0.99,0.87,0.87,0.99,0.81,0.98,0.73,0.92,0.99,...,0.76,0.96,0.95,0.73,0.69,0.92,0.98,0.97,0.96,0.86
C0003,0.98,0.87,1.00,0.99,0.92,0.99,0.94,0.96,0.98,0.87,...,0.98,0.96,0.97,0.96,0.95,0.99,0.93,0.74,0.96,0.99
C0004,0.98,0.87,0.99,1.00,0.92,0.98,0.94,0.96,0.97,0.88,...,0.97,0.96,0.97,0.96,0.95,0.98,0.92,0.74,0.96,0.99
C0005,0.97,0.99,0.92,0.92,0.99,0.87,0.99,0.80,0.96,0.99,...,0.83,0.98,0.98,0.80,0.77,0.96,0.99,0.94,0.99,0.91


In [64]:
#According to the deliverable demands:the top 20 customers in ASC order are identified and stored
top_20_customers = customer_records['CustomerID'].head(20)
# Initialize a list to store the lookalikes in the required format
lookalike_data = []
for customer in top_20_customers:
    # Get the similar customers and their similarity scores, sorted in descending order (ignoring self)
    similar_customers = similarity_dataset[customer].drop(customer).sort_values(ascending=False).head(3)
    # Create the list of tuples (lookalike, similarity)
    lookalike_list = [(similar_customer, round(similarity, 2)) for similar_customer, similarity in zip(similar_customers.index, similar_customers.values)]
    # Append the customer ID and the lookalike list to the data
    lookalike_data.append([customer, lookalike_list])
# Convert the result into a DataFrame
lookalike_dataset = pd.DataFrame(lookalike_data, columns=['CustomerID', 'Lookalikes'])




In [66]:
lookalike_dataset.shape

(20, 2)

In [77]:
from google.colab import files
# Save as CSV file
lookalike_dataset.to_csv('Lookalike_dataset.csv', index=False)
